---
jupyter:
  jupytext:
    formats: md,ipynb
    text_representation:
      extension: .md
      format_name: markdown
      format_version: '1.3'
      jupytext_version: 1.16.0
  kernelspec:
    display_name: Python 3 (ipykernel)
    language: python
    name: python3
---

<!-- #region id="496978f4" -->
# Table des matières
1. [Lecture des données](#lecture-des-données)
1. [Premier pipeline](#première-pipeline)
    1. [Définition du pipeline](#définition-du-pipeline)
    1. [Entraînement du pipeline avec les données d'entraînement](#entraînement-du-pipeline-avec-les-données-dentraînement)
      1. [Important](#important)
    1. [Évaluation de ses performances en classification sur l'ensemble de test](#évaluation-de-ses-performances-en-classification-sur-lensemble-de-test)
1. [Deuxième pipeline](#deuxième-pipeline)
    1. [Définition du pipeline](#définition-du-pipeline-1)
    1. [Définition de la recherche sur grille](#définition-de-la-recherche-sur-grille)
    1. [Entraînement du pipeline avec chaque combinaison de paramètres](#entraînement-du-pipeline-avec-chaque-combinaison-de-paramètres)
    1. [Affichage des paramètres du pipeline optimal](#affichage-des-paramètres-du-pipeline-optimal)
1. [Troisième pipeline](#troisième-pipeline)
    1. [Définition du pipeline](#définition-du-pipeline-2)
    1. [Définition de la recherche sur grille](#définition-de-la-recherche-sur-grille)
    1. [Entraînement du pipeline avec chaque combinaison de paramètres](#entraînement-du-pipeline-avec-chaque-combinaison-de-paramètres-1)
    1. [Affichage des paramètres du pipeline optimal](#affichage-des-paramètres-du-pipeline-optimal-1)
1. [Diviser pour conquérir](#diviser-pour-conquérir)
<!-- #endregion -->



In [ ]:
import os
import sys
import warnings

import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, decomposition, discriminant_analysis, manifold
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PowerTransformer, QuantileTransformer, StandardScaler
from sklearn.svm import SVC

# Élimination des avertissements (warnings). On en observe régulièrement lorsqu'on utilise
# une méthode de recherche d'hyperparamètres. Plusieurs des entraînements associés
# ne convergent pas et un message d'avertissement est affiché pour chacun.
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore"

seed = 42

rng_seed = seed



<!-- #region id="6ba1bb53" -->
On a vu, dans les modules précédents sur le prétraitement des données, les différentes étapes de
nettoyage, d'imputation de données manquantes, de normalisation et de redimensionnement.
Dans chaque cas, on étudiait l'étape séparément,
hors d'un contexte global, afin d'apprendre à l'utiliser dans des situations précises.
Lorsqu'on finit par maîtriser chaque étape, il devient intéressant de pouvoir
en combiner plusieurs, sinon toutes, afin d'élaborer un processus de prétraitement plus spécialisé.

Imaginons un projet où l'on doit effectuer chacune des opérations de prétraitement apparaissant dans
la figure suivante avant même de passer à la phase de classification (de régression, de regroupement de données, etc.).
Il nous faudrait optimiser chacune des étapes en tenant compte des autres afin de maximiser les performances
de notre application finale.
<!-- #endregion -->

<!-- #region id="561d3d77" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/data-processing.png"  width="500" />
    <div>
    <font size="1.5">Image Source: http://pzs.dstu.dp.ua/DataMining/preprocessing/bibl/Data%20Preprocessing%20in%20Data%20Mining.pdf/</font>
    </div>
</div>
<p>&nbsp;</p>
<!-- #endregion -->

<!-- #region id="bfee40f1" -->
C'est exactement ce que permet de faire l'optimisation d'un pipeline d'analyse des données! La majorité des projets
devraient éventuellement passer par cette étape.
Elle est efficace sur le plan de la gestion et du développement d'un projet complexe. Idéalement,
on devrait ne modifier que quelques sections d'un pipeline lorsqu'on veut tester une nouvelle idée
ou implémenter un nouvel algorithme d'analyse. Pas besoin de recommencer un nouveau programme à
chaque fois; la roue une fois inventée n'a besoin que d'être poussée dans une nouvelle direction.

Le concept de pipeline d'analyse des données est un des joyaux de la librairie Scikit-learn, rien de moins. Nous
allons apprendre comment en construire plusieurs et comment les optimiser pour une tâche donnée. Un pipeline
simplifie considérablement la tâche des programmeurs. Apprenez à les utiliser!
<!-- #endregion -->

<!-- #region id="70fa1d81" -->
# <a id=lecture-des-données>Lecture des données</a>
<!-- #endregion -->

<!-- #region id="ab120b30" -->
Nous allons utiliser à nouveau un sous-ensemble du jeu de données
[**MNIST**](https://en.wikipedia.org/wiki/MNIST_database)
qui comprend des images de chiffres 0 à 9 de taille $8 \text{ par } 8$. Les images du jeu de données original sont de taille $28 \text{ par } 28$. Nous allons n'utiliser que les images des chiffres 0 à 5.
<!-- #endregion -->



In [ ]:
# Lecture du jeu de données et séparation de celles-ci en ensembles d'entraînement et de test

digits = datasets.load_digits(n_class=6)
X = digits.data
y = digits.target
n_samples, n_features = X.shape
n_neighbors = 30
n_components = 2

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=rng_seed
)


In [ ]:
# Affichage de quelques images de chiffres pour chaque classe (0 à 5)

n_img_per_row = 20
img = np.zeros((10 * n_img_per_row, 10 * n_img_per_row))
for i in range(n_img_per_row):
    ix = 10 * i + 1
    for j in range(n_img_per_row):
        iy = 10 * j + 1
        img[ix : ix + 8, iy : iy + 8] = X[i * n_img_per_row + j].reshape((8, 8))

plt.imshow(img, cmap=plt.cm.binary)
plt.xticks([])
plt.yticks([])
plt.title("Une sélection des 6 premiers chiffres du jeu de données MNIST")



<!-- #region id="16eab83e" -->
# <a id=première-pipeline>Premier pipeline</a>
<!-- #endregion -->

<!-- #region id="9ab94536" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/pipeline-photo.jpeg"  width="400" />
    <div>
    <font size="1.5">Image Source: https://www.bdcmagazine.com/2020/08/the-importance-of-pipeline-repair-services//</font>
    </div>
</div>
<p>&nbsp;</p>
<!-- #endregion -->

<!-- #region id="ecd550ce" -->
Comme premier essai, construisons un pipeline de base combinant deux étapes de prétraitement, suivie d'une étape
de classification. On utilise ici un classificateur de type
[SVM](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) non vu dans la formation.
Le type de classificateur n'est pas vraiment important dans cette série d'exemples. D'ailleurs, il
recommandé d'en essayer plusieurs lors d'un projet et de choisir le meilleur parmi eux. Dans le cas du classificateur SVM,
il a deux hyperparamètres à ajuster: $\alpha$ et $C$.

Nous allons effectuer les étapes suivantes:

- normalisation standard,
- redimensionnement avec la méthode PCA,
- classification avec la méthode SVM. 

Chaque opération utilisera ses paramètres par défaut ou ceux spécifiés dans l'appel de la fonction.
<!-- #endregion -->

<!-- #region id="3332b350" -->
## <a id=définition-du-pipeline-1>Définition du pipeline</a>
<!-- #endregion -->

<!-- #region id="e78eabf0" -->
Les noms (« `"transformation"` », « `"reduce_dim"` », « `"classify"` ») utilisés dans la définition du pipeline sont arbitraires. Vous
pouvez les changer, mais faites en sorte qu'ils signifient quelque chose de plus utile que « testA », « Bozo », ou autres noms avec peu de signification.

Après chaque nom apparaît le nom de l'opérateur associé, par exemple `'StandardScaler'` qui effectue la
normalisation standard. Puis, apparaissent les hyperparamètres sélectionnés différents de ceux par
défaut de chaque opérateur.

L'opérateur de réduction de dimensionnalité indique qu'on ne garde que les deux premières composantes
après la transformation en composantes principales.

L'opérateur de classification SVC indique qu'on utilise un classificateur de type SVM avec quatre
hyperparamètres d'initialisations.
<!-- #endregion -->



In [ ]:
# Définition du pipeline

pipeline = Pipeline(
    [
        ("transformation", StandardScaler()),
        ("reduce_dim", decomposition.PCA(n_components=2)),
        ("classify", SVC(kernel="rbf", C=1, gamma=0.2, max_iter=1000)),
    ]
)



<!-- #region id="2d0df9a4" -->
## <a id=entraînement-du-pipeline-avec-les-données-dentraînement>Entraînement du pipeline avec les données d'entraînement</a>
<!-- #endregion -->

<!-- #region id="fd5c857b" -->
Cette opération calcule les paramètres suivants:


- coefficients de la transformation de normalisation standard,
- coefficients de la matrice de conversion en composantes principales
ne tenant compte que des deux premières composantes,
- coefficients du classificateur SVM.


Une fois entrainés, les différents opérateurs pourront être appliqués à de nouvelles données, celles de test.

<!-- #endregion -->



In [ ]:
pipeline.fit(X_train, y_train)



<!-- #region id="eb6f93cb" -->
### <a id=important>Important</a>
<!-- #endregion -->

<!-- #region id="78652762" -->
Si l'on désire traiter un nouvel ensemble de données `X_new`, il ne reste plus qu'à faire ceci:



In [ ]:
y_pred = pipeline.predict(X_new)



Si l'on désire calculer le score associé, on fait ceci:



In [ ]:
score = pipeline.score(X_new, y_new)


<!-- #endregion -->

<!-- #region id="24c7737d" -->
## <a id=évaluation-de-ses-performances-en-classification-sur-lensemble-de-test>Évaluation de ses performances en classification sur l'ensemble de test</a>
<!-- #endregion -->



In [ ]:
print("\nScore en entraînement: %0.1f %%" % (100 * pipeline.score(X_train, y_train)))
print("\nScore en test: %0.1f %%" % (100 * pipeline.score(X_test, y_test)))



<!-- #region id="ecfff660" -->
# <a id=deuxième-pipeline>Deuxième pipeline</a>
<!-- #endregion -->

<!-- #region id="7d426a7b" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/pipeline-photo2.jpeg"  width="400" />
    <div>
    <font size="1.5">Image Source: https://pxhere.com/en/photo/1081470/</font>
    </div>
</div>
<p>&nbsp;</p>
<!-- #endregion -->

<!-- #region id="c13af654" -->
Supposons maintenant qu'on ne soit pas certain que la normalisation standard soit la meilleure méthode de
transformation des données. On aimerait en tester au moins deux:


- normalisation standard,
- transformation par les quantiles (`QuantileTransformer`).


On voudrait garder inchangées les étapes précédentes de redimensionnement et de classification.
<!-- #endregion -->

<!-- #region id="16c2984e" -->
## <a id=définition-du-pipeline-2>Définition du pipeline</a>
<!-- #endregion -->

<!-- #region id="89d7d6e0" -->
Puisqu'il y a deux méthodes de transformation de données, la grille des paramètres suivante contient deux dictionnaires de paramètres. On peut ainsi considérer qu'il y a deux mini pipelines; un pour chaque méthode de transformation. On procède en utilisant le mot clé `'passthrough'` qui permet de spécifier chacun des choix à essayer pour l'étape de transformation des données.
<!-- #endregion -->



In [ ]:
pipeline = Pipeline(
    [
        ("transformation", "passthrough"),
        ("reduce_dim", decomposition.PCA(n_components=2)),
        ("classify", SVC(kernel="rbf", C=1, gamma=0.2, max_iter=1000)),
    ]
)

# Grille des paramètres à modifier. le mot clé passthrough renvoie ici
param_grid = [
    {"transformation": [StandardScaler()]},
    {
        "transformation": [
            QuantileTransformer(output_distribution="normal", n_quantiles=50)
        ]
    },
]



<!-- #region id="90c10d06" -->
Il faut maintenant tester les deux mini pipelines afin de trouver le plus performant des deux. Pour
cela, on utilisera la fonction
[`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV.predict)
qui permet d'effectuer une recherche sur une grille d'hyperparamètres qui sont
dans ce cas-ci les deux méthodes de transformation.
<!-- #endregion -->

<!-- #region id="208e9e6a" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/light-bulb-idea.jpeg"  width="200" />
    <div>
    <font size="0.5">Image Source: https://basementdesigner.com/basement-finishing-102/light-bulb-idea//</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="57284d80" -->
`GridSearchCV` procède par validation croisée sur l'ensemble d'entraînement. Les principes de la validation
croisée sont expliqués dans la partie 5. En gros, elle permet de choisir le plus performant parmi
plusieurs modèles, dans ce cas-ci, entre les deux mini pipelines.

Enfin, `GridSearchCV` réentraîne le pipeline optimal en utilisant cette fois-ci toutes les données
d'entraînement, sans validation croisée.


Si l'on désire traiter un nouvel ensemble de données `X_new`, il ne reste plus qu'à faire ceci:




In [ ]:
y_pred = grid_search.predict(X_new)



Si l'on désire calculer le score associé, on fait ceci:



In [ ]:
score = grid_search.score(X_new, y_new)


<!-- #endregion -->

<!-- #region id="4772afd4" -->
## <a id=définition-de-la-recherche-sur-grille>Définition de la recherche sur grille</a>
<!-- #endregion -->



In [ ]:
grid_search = GridSearchCV(
    pipeline, param_grid=param_grid, n_jobs=2, verbose=1, refit=True
)



<!-- #region id="27c28bed" -->
## <a id=entraînement-du-pipeline-avec-chaque-combinaison-de-paramètres-1>Entraînement du pipeline avec chaque combinaison de paramètres</a>
<!-- #endregion -->

<!-- #region id="010e75b1" -->
L'affichage qui suit indique que 10 *fits* (entraînement) ont été effectués. Il y avait 2 modèles à
tester et 5 repliements pour chacun lors de la validation croisée, soit un total de 10 entraînements.
<!-- #endregion -->



In [ ]:
grid_search.fit(X_train, y_train)



<!-- #region id="d29f11b1" -->
## <a id=affichage-des-paramètres-du-pipeline-optimal-1>Affichage des paramètres du pipeline optimal</a>
<!-- #endregion -->

<!-- #region id="f1e42289" -->
Définissons une fonction permettant d'afficher les paramètres.
<!-- #endregion -->



In [ ]:
def afficheMeilleurChoixParametres(grid_search):
    print("\nMeilleur choix de paramètres:")
    steps = dict(grid_search.best_estimator_.steps)

    for param_name in sorted(steps.keys()):
        print("\t%s: %r" % (param_name, steps[param_name]))

    print("\nScore optimal en entraînement: %0.1f %%" % (100 * grid_search.best_score_))
    print(
        "\nScore en test avec le pipeline optimal: %0.1f %%\n"
        % (100 * grid_search.score(X_test, y_test))
    )


In [ ]:
afficheMeilleurChoixParametres(grid_search)



<!-- #region id="532001c3" -->
# <a id=troisième-pipeline>Troisième pipeline</a>
<!-- #endregion -->

<!-- #region id="3942feef" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/pipeline-photo3.jpeg"  width="400" />
    <div>
    <font size="1.5">Image Source: https://www.wallpaperflare.com/search?wallpaper=pipeline/</font>
    </div>
</div>
<p>&nbsp;</p>

<!-- #endregion -->

<!-- #region id="b08a5b62" -->
Se pourrait-il que le choix précédent de la normalisation standard, par rapport à l'autre méthode de transformation, ait
été influencé par les valeurs des paramètres des deux autres opérations? C'est très pertinent
comme question. On va le vérifier.

On va optimiser à nouveau le pipeline en faisant varier les facteurs suivants:


- Transformation:
   - normalisation standard,
   - transformation par les quantiles (`QuantileTransformer`). 
- Réduction de la dimensionnalité:
    - `n_components` $= [2, 4, 8, 16, 32]$
- Classificateur SVM:
    - `C` $= [0.1, 1, 10, 100, 1000]$,
    - `gamma` $= [1, 0.1, 0.01, 0.001, 0.0001]$.


<!-- #endregion -->

<!-- #region id="d520bf6f" -->
## <a id=définition-du-pipeline>Définition du pipeline</a>
<!-- #endregion -->

<!-- #region id="999d21ec" -->
Puisqu'il y a à nouveau deux méthodes de transformation de données, la grille des paramètres suivante contient deux dictionnaires de paramètres. Il y a encore deux mini pipelines.
<!-- #endregion -->



In [ ]:
pipeline = Pipeline(
    [
        ("transformation", "passthrough"),
        ("reduce_dim", decomposition.PCA()),
        ("classify", SVC(kernel="rbf", max_iter=1000)),
    ]
)

param_grid = [
    {
        "transformation": [StandardScaler()],
        "reduce_dim__n_components": [2, 4, 8, 16, 32],
        "classify__C": [0.1, 1, 10, 100, 1000],
        "classify__gamma": [1, 0.1, 0.01, 0.001, 0.0001],
    },
    {
        "transformation": [
            QuantileTransformer(output_distribution="normal", n_quantiles=50)
        ],
        "reduce_dim__n_components": [2, 4, 8, 16, 32],
        "classify__C": [0.1, 1, 10, 100, 1000],
        "classify__gamma": [1, 0.1, 0.01, 0.001, 0.0001],
    },
]



<!-- #region id="4d68f200" -->
## <a id=définition-de-la-recherche-sur-grille>Définition de la recherche sur grille</a>
<!-- #endregion -->



In [ ]:
grid_search = GridSearchCV(
    pipeline, param_grid=param_grid, n_jobs=2, verbose=1, refit=True
)



<!-- #region id="c7ac6f3c" -->
## <a id=entraînement-du-pipeline-avec-chaque-combinaison-de-paramètres>Entraînement du pipeline avec chaque combinaison de paramètres</a>
<!-- #endregion -->

<!-- #region id="4614cd6d" -->
L'affichage qui suit indique que 1 250 entraînements ont été effectués. Il y avait 2 modèles à tester, $5 \times 5 \times 5=125$
combinaisons d'hyperparamètres pour chacun, et 5 repliements pour chacun lors de la validation croisée,
soit un total de $2\times 125\times 5= 1\ 250$ entraînements.

> Prends un peu de temps à être exécuter étant donné qu'il y a 1 250 entraînements à faire.
<!-- #endregion -->



In [ ]:
grid_search.fit(X_train, y_train)



<!-- #region id="c28d08fc" -->
## <a id=affichage-des-paramètres-du-pipeline-optimal>Affichage des paramètres du pipeline optimal</a>
<!-- #endregion -->



In [ ]:
afficheMeilleurChoixParametres(grid_search)



<!-- #region id="80511ee6" -->
Le score en test est passé de $83,8~\%$ à $98,9~\%$!
<!-- #endregion -->

<!-- #region id="4592e89f" -->
# <a id=diviser-pour-conquérir>Diviser pour conquérir</a>
<!-- #endregion -->

<!-- #region id="f37bb5a2" -->
Bien qu'il soit pratique de nettoyer ses données et de les classifier dans un même pipeline, ce
n'est généralement pas la meilleure stratégie.

Vous devriez plutôt utiliser un pipeline pour trouver la bonne méthode de prétraitement puis l'implémenter.
Ça vous permettrait de vous concentrer ensuite sur la partie la plus intéressante de votre projet c.-à-d. la
classification, la régression, etc. Créez un second pipeline pour traiter cette nouvelle partie
d'un projet. Inutile de refaire le prétraitement des données à chaque fois.

<!-- #endregion -->
